# 1) Relación Entre Viajes En Bicicleta y el Clima

In [1]:
import numpy as np
import pandas as pd

import seaborn as sbn
import matplotlib.pyplot as plt
%matplotlib inline

import datetime
import calendar

In [4]:
trips = pd.read_csv('data/trip.csv', low_memory=False)
weather = pd.read_csv('data/weather.csv', low_memory=False)
#Se convierte los dates a datetime64[ns].
trips['start_date'] = pd.to_datetime(trips['start_date'])
weather['date'] = pd.to_datetime(weather['date'])

In [ ]:
#Se agrega una nueva columna date que coincide con weather.
trips['date'] = trips['start_date'].apply(lambda x: x.date())
#Se convierte date a datetime64[ns].
trips['date'] = pd.to_datetime(trips['date'])

In [ ]:
#Se convierte zip_code a string en weather para coincidir con trips.
#NOTA: No se convierte zip_code de trips a int64 por un extraño error en to_numeric.
weather['zip_code'] = weather['zip_code'].astype(str)

In [ ]:
#Formula para convertir F a C.
def f_to_c(f_temp):
    return round((f_temp - 32) / 1.8, 2)

In [ ]:
#Se crean columnas con las temperaturas en C.
weather['max_temperature_c'] = weather['max_temperature_f'].map(f_to_c)
weather['mean_temperature_c'] = weather['mean_temperature_f'].map(f_to_c)
weather['min_temperature_c'] = weather['min_temperature_f'].map(f_to_c)

In [ ]:
#Se crean columnas con visibilidad en Km.
weather['max_visibility_km'] = weather['max_visibility_miles'].map(lambda x: x * 1.6)
weather['mean_visibility_km'] = weather['mean_visibility_miles'].map(lambda x: x * 1.6)
weather['min_visibility_km'] = weather['min_visibility_miles'].map(lambda x: x * 1.6)

In [ ]:
#Se combinan los Dataframes.
joined = trips.merge(weather, left_on=['date', 'zip_code'], right_on=['date', 'zip_code'])

In [ ]:
#Funcion para convertir la duracion de segundos a minutos.
def s_to_m(time):
    return (time / 60)
#Funcion para convertir la duracion de segundos a horas redondeo a 3 decimales.
def s_to_h(time):
    return round((time / 3600),3)

In [ ]:
#Se crea una columna con la duracion en minutos y la duracion en horas.
joined['duration_m'] = joined['duration'].map(s_to_m)
joined['duration_h'] = joined['duration'].map(s_to_h)

## 1.1) Viajes y Temperatura

En esta serie de plots se analizará si hay una correlación entre los viajes en bicicleta y la temperatura.

Como los registros del clima están en forma diurna, es necesario tomar las fechas del viaje sin horarios. Para todos los plots se toma como fecha del viaje a la fecha de inicio del mismo, esto se debe a que es el momento en el cual la persona toma en cuenta las condiciones climáticas para decidir si realizar un viaje o no. 

### 1.1.A) Histograma del Promedio de Temperatura

In [ ]:
joined.hist(column='mean_temperature_c', grid=True, figsize=(10,10), xrot=90);
plt.xticks(range(10,24,1));

En este histograma se puede apreciar que la mayor cantidad de los viajes se realizan cuando la temperatura promedio esta entre 12°C y 21°C.  
Si bien puede parecer que la temperatura es algo baja, hay que tener en cuenta que esto es un promedio de la temperatura de todo el día y como San Francisco es una ciudad costera la temperatura suele bajar bastante sobre la noche.

### 1.1.B) Histograma de la Temperatura Máxima

In [ ]:
joined.hist(column='max_temperature_c', grid=True, figsize=(10,10));
plt.xticks(range(15,25,1));

De este histograma se puede obtener que la mayoria de los viajes se realizan cuando la temperatura maxima esta entre 18°C y 23°C.

Si se toman en cuenta los dos histogramas en conjunto, se puede apreciar que la mayoría de las personas buscan temperaturas templadas a la hora de realizar los viajes.  
No es necesario analizar la temperatura mínima, ya que como se dijo antes, la temperatura suele bajar mucho sobre la noche y daría resultados engañosos.

### 1.1.C) Dias con mayor uso simultaneo de Bicicletas y la Temperatura

In [ ]:
#Funciones para clasificar.
def f_st(row):
    if row['event'] == 'start_date':
        val = 1
    else:
        val = 0
    return val

def f_en(row):
    if row['event'] == 'end_date':
        val = 1
    else:
        val = 0
    return val

In [ ]:
trips_aux = trips[['id', 'start_date', 'end_date', 'zip_code']]
trips_melt = pd.melt(trips_aux, id_vars=['id','zip_code'], value_vars=['start_date', 'end_date'], var_name='event', value_name='time')
trips_melt['time'] = pd.to_datetime(trips_melt['time'])

In [ ]:
#Se obtiene la cantidad de bicicletas en uso al mismo tiempo.
trips_ord = trips_melt.sort_values('time', ascending=True) 
trips_ord['start_counter'] = trips_ord.apply(f_st, axis=1)
trips_ord['end_counter'] = trips_ord.apply(f_en, axis=1)
trips_ord['start'] = trips_ord['start_counter'].cumsum()
trips_ord['end'] = trips_ord['end_counter'].cumsum()
trips_ord = trips_ord[['id', 'zip_code', 'time', 'start', 'end']]
trips_ord['in_use'] = trips_ord['start'] - trips_ord['end']
trips_ord = trips_ord.sort_values('in_use', ascending=False)

In [ ]:
#Se eliminan los horarios para coincidir con weather.csv.
trips_ord['time'] = trips_ord['time'].apply(lambda x: x.date())
#Se convierte time a datetime64[ns].
trips_ord['time'] = pd.to_datetime(trips_ord['time'])

In [ ]:
#Se combinan los Dataframes.
joined_simul = trips_ord.merge(weather, left_on=['time', 'zip_code'], right_on=['date', 'zip_code'])

In [ ]:
#Solo hay que quedarse con el maximo de bicicletas simultaneas para ese dia.
joined_max_simul = joined_simul.drop_duplicates(subset=['time'], keep='first')

In [ ]:
#Nos quedamos con los 10 valores maximos y las columnas que interesan.
joined_max_simul_bar = joined_max_simul[:10]
joined_max_simul_bar = joined_max_simul_bar[['time', 'mean_temperature_c', 'max_temperature_c']]
joined_max_simul_bar.set_index('time', inplace=True)

En el siguiente gráfico de barras se pueden aprecian los 10 días (de los cuales hay datos de temperatura) con mayor uso simultaneo de bicicletas(ordenados de forma descendente), junto con su temperatura promedio y temperatura máxima. La locación tomada para el clima de cada día, es donde se produce su respectivo pico máximo de bicicletas en uso.

In [ ]:
bar = joined_max_simul_bar.plot.bar(figsize=(10,10), fontsize=20);
#Elimina el 00:00:00 del plot.
bar.set_xticklabels(joined_max_simul_bar.index.format());
plt.yticks(range(0,33,2));

Primero es interesante notar que si bien el día con mayor uso simultaneo es también el día con mayor temperatura máxima, unos 31°C, los días restantes no tienen temperaturas máximas tan elevadas.  
El segundo punto importante a notar es que la mayoría de estos días tienen temperaturas templadas, es decir, sus temperaturas promedio varían entre 14°C y 23°C mientras que sus temperaturas máximas están entre 19 y 25°C.  
Por ultimo, es evidente que si bien la cantidad de bicicletas en uso simultáneamente decrece a lo largo del gráfico de barras, las temperaturas se mantienen siempre dentro de un mismo rango.

### 1.1.D) Conclusión de Viajes y Temperatura

Usando los plots anteriores se puede concluir que **la mayor cantidad de viajes se realizan cuando la temperatura es templada**, es decir, cuando esta **alrededor de los 25°C**.
Para describir este fenómeno hay que tener en cuenta los motivos por los cuales se podría realizar un viaje, para simplificar se tomaran dos casos:
1. En el caso de que se quiera realizar un viaje por la necesidad de trasladarse, la temperatura no afecta mucho, excepto en casos de muy bajas o muy altas temperaturas.
2. El caso en el cual se quiere realizar un viaje por placer es el que interesa, ya que es aquí cuando la temperatura juega un rol importante. Con temperaturas templadas las personas estarán mas predispuestas a realizar viajes en bicicleta, como muestran los plots obtenidos. 

Por ultimo, es importante notar que las mejores temperaturas se suelen presentar sobre la tarde, ya que es cuando comienza a bajar el sol, por eso no seria extraño que la mayor cantidad de los viajes se realicen en horarios de la tarde.

## 1.2) Viajes y Visibilidad

En la siguiente serie de plots se tratara de analizar si hay una correlación entre los viajes en bicicletas y la visibilidad.  

Como los registros del clima están en forma diurna, es necesario tomar las fechas del viaje sin horarios. Para todos los plots se toma como fecha del viaje a la fecha de inicio del mismo, esto se debe a que es el momento en el cual la persona toma en cuenta las condiciones climáticas para decidir si realizar un viaje o no. 

### 1.2.A)  Grafico de Barras de Visibilidad Promedio y Histograma de Visibilidad Promedio

In [ ]:
joined.hist(column='mean_visibility_km', grid=True, figsize=(10,10), xrot=90);
plt.xticks(range(10,20,1));

In [ ]:
joined['mean_visibility_km'].value_counts(sort=True).plot.bar(figsize=(10,10));

De este plot se pueden extraer tres puntos importantes:  
* La mayoría de los viajes en bicicleta se realizan cuando la visibilidad promedio esta entre 14.4km y 16km, mas precisamente cuando esta en 16km.
* Por encima de los 16km de visibilidad promedio hay escasez de datos, por lo tanto, se puede intuir que no es normal que en San Francisco se presenten visibilidades promedio tan elevadas.  
* Sino se toman en cuenta las visibilidades promedio mayores a 16km(porque no hay datos suficientes), la diferencia entre las visibilidades promedio mayores o iguales a 14.4km y las menores a esta es abismal. Esto no es un dato menor, ya que hay que tener una buena visibilidad al realizar un viaje para que este sea seguro y queda claro, por esta diferencia, que el usuario prioriza mucho la seguridad.

Ahora seria importante analizar que sucede con la visibilidad mínima, ya que por cuestiones de seguridad es importante que haya una buena visibilidad mínima para evitar accidentes.

### 1.2.B) Grafico de Barras de Visibilidad Mínima y Histograma de Visibilidad Mínima

In [ ]:
joined.hist(column='min_visibility_km', grid=True, figsize=(10,10), xrot=90, bins=20);
plt.xticks(range(0,21,1));

In [ ]:
joined['min_visibility_km'].value_counts(sort=True).plot.bar(figsize=(10,10));

En estos dos plots se apreciar de nuevo que la mayor cantidad de viajes se realiza cuando la visibilidad mínima es de 16km. Ademas es importante notar que de nuevo se manifiesta la gran separación entre la visibilidad de 16km y las que son menores a esta. Por ejemplo, si tomamos la visibilidad mínima de 11.2km podemos ver que hay una diferencia aproximada de 40000 viajes con la visibilidad de 16km

### 1.2.C) Grafico de Barras de Visibilidad Máxima y Histograma de Visibilidad Máxima

In [ ]:
joined['max_visibility_km'].value_counts(sort=True).plot.bar(figsize=(10,10));

De este plot es muy difícil sacar conclusiones ya que la diferencia de la visibilidad máxima de 16km con las demás es demasiado grande. Lo único importante a destacar, es que no hay ningún valor de visibilidad máxima menor a 9.6km, lo cual indicaría que nunca hay visibilidades máximas menores a 9.6km, lo cual es poco probable, o bien que no se realizan viajes cuando la visibilidad máximas son tan bajas.

### 1.2.D) Dias con mayor uso simultaneo de Bicicletas y la Visibilidad

In [ ]:
#Nos quedamos con los 10 valores maximos y las columnas que interesan.
joined_max_simul_vis_bar = joined_max_simul[:10]
joined_max_simul_vis_bar = joined_max_simul_vis_bar[['time', 'mean_visibility_km', 'min_visibility_km', 'max_visibility_km']]
joined_max_simul_vis_bar.set_index('time', inplace=True)

En el siguiente gráfico de barras se pueden aprecian los 10 días (de los cuales hay datos de temperatura) con mayor uso simultaneo de bicicletas(ordenados de forma descendente), junto con su visibilidad promedio, visibilidad mínima y visibilidad máxima. La locación tomada para el clima de cada día, es donde se produce su respectivo pico máximo de bicicletas en uso.

In [ ]:
bar = joined_max_simul_vis_bar.plot.bar(figsize=(10,10), fontsize=20);
#Elimina el 00:00:00 del plot.
bar.set_xticklabels(joined_max_simul_vis_bar.index.format());
plt.yticks(range(0,17,4));

Este plot confirma la tendencia que se había marcado previamente, los 10 dias con mayor uso simultaneo de bicicletas presentan una visibilidad promedio de 16km. Ademas la mayoría presentan visibilidades mínimas y máximas también de 16km.  
El único caso llamativo es del dia 2014-10-20 en el cual la visibilidad mínima es de 3km, pero si bien su visibilidad mínima es baja su visibilidad máxima y visibilidad promedia se mantienen dentro de lo esperado. Esto nos indicaría que la mayor cantidad de viajes de ese día se habría producido en horarios donde la visibilidad era más cercana a la máxima que a la mínima.

### 1.2.E) Conclusión de Viajes y Visibilidad

Dados los plots vistos se puede concluir claramente que la visibilidad máxima, mínima y mediana deben ser de, o al menos cercano a, 16km para que se produzca un mayor uso del servicio de bicicletas. Esto claramente esta relacionado, como ya se menciono antes, con la seguridad, los usuarios ponen a la seguridad como uno de los puntos más importantes a la hora de decidir si realizar un viaje en bicicleta o no. Por ejemplo, nadie realizaría un viaje bicicleta cuando hay niebla(evento en el cual hay una visibilidad muy baja) porque eso conllevaría un riesgo importante. 

## 1.3) Viajes y Estaciones Climáticas

### 1.3.A) Cantidad de Viajes y Estaciones Climáticas

En la siguiente serie de plots se analizará en que estación climática se realizan más viajes.

In [ ]:
#Funcion para clasificar estaciones climaticas.
def estacion(date):
    if date.month >= 3 and date.month <= 5:
        return 'Primavera'
    elif date.month >= 6 and date.month <= 8:
        return 'Verano'
    elif date.month >= 9 and date.month <= 11:
        return 'Otoño'
    else:
        return 'Invierno'

In [ ]:
#Se crea la columna con la estacion climatica.
joined['estacion_clima'] = joined['date'].map(estacion)

In [ ]:
joined['estacion_clima'].value_counts(sort=True).plot.bar(figsize=(10,10), rot=0);
plt.yticks(range(0,26000,1000));

De este plot podemos observar:  
* La mayoría de los viajes se realizan en Primavera, esto concuerda con lo visto en la sección de temperatura ya que en Primavera se presentan temperaturas cercanas a los 25°C.
* La segunda estación climática es el Otoño, esto vuelve a concordar con lo visto en la sección de temperatura ya que, si bien las temperaturas suelen ser mas frías que en Primavera, presenta temperaturas cercanas a los 15°C.
* Por ultimo, es importante notar que todas las estaciones mantienen un piso de 20000 viajes. Esto va de la mano con lo dicho en la sección 1.D y 2.E, si alguien quiere utilizar el servicio de bicicletas por un motivo no recreativo, no suele darle demasiada importancia a factores climáticos excepto que conlleven un riesgo para su seguridad.

### 1.3.B) Duración promedio de Viajes y Estaciones Climáticas

En este plot se analizará las duraciones promedio de los viajes en cada estación.

In [ ]:
grouped_season = joined[['estacion_clima', 'duration_m']].groupby('estacion_clima')\
.aggregate('mean')

In [ ]:
grouped_season.plot.bar(figsize=(10,10), rot=0);
plt.yticks(range(0,15,1));

De este plot podemos hacer dos observaciones importantes:  
* Primero, la Primavera, que tiene la mayor cantidad de viajes, es la que posee la menor duración promedio de viajes.  
* Segundo, todas las estaciones tienen una duración promedio de viajes similares.

### 1.3.C) Conclusión de Viajes y Estaciones Climáticas

La conclusión más importante que se puede obtener de los plots analizados es que la diferencia entre las diferentes estaciones es muy pequeña. Esto se podría deber a los siguientes motivos:  
* Como se expreso anteriormente, todas las estaciones tienen un piso 20000 viajes. Esto da a entender que una gran parte de los usuarios no pone como prioridad al clima a la hora de realizar un viaje, esto se puede deber a que la mayoría de los usuarios utiliza el servicio de bicicletas como modo de transporte y no como en forma recreativa.  
* Si bien los climas varían entre estaciones, los cambios no son tan drásticos como para generar una diferencia sustancial entre la cantidad y duración de viajes en bicicleta entre cada estación.

# 1.4) Caso Particular: 5 Viajes con Mayor Duración

En los siguientes plots se analizara que sucede con la temperatura, visibilidad y estación climática para los 5 viajes con mayor duración. Se utilizan las duraciones menores o iguales a 12 horas porque se considera que un viaje continuado puede durar, como mucho, 12 horas si se permiten interrupciones de duración corta. Algunos de los motivos por los cuales se pueden realizar viajes largos son, por ejemplo:  
* Realizar un tour de la ciudad de San Francisco.  
* Realizar actividad física intensiva.
* Tener que realizar varios trayectos en horarios diferentes.

In [ ]:
#Se obtienen los 5 viajes con mayor duracion (que poseen datos climaticos) menores a 12 horas.
top_dur = joined[joined['duration_h'] <= 12].sort_values('duration_h', ascending=False)[:5]

### 1.4.A) Temperatura de los 5 Viajes con Mayor Duración

In [ ]:
top_dur_temp = top_dur[['duration_h', 'max_temperature_c', 'min_temperature_c', 'mean_temperature_c']]
top_dur_temp.set_index('duration_h', inplace=True)

Se muestran los 5 viajes con mayor duración (con sus respectivas duraciones, en horas, como label) ordenados de forma descendente junto con sus respectivas temperaturas máximas, temperaturas mínimas y temperaturas promedio.

In [ ]:
top_dur_temp_bar = top_dur_temp.plot.bar(figsize=(10,10), fontsize=20, rot=0);
plt.yticks(range(0,30,2));

De este plot se puede hacer las siguientes observaciones:  
* El viaje con mayor duración de los 5 es el que posee la temperatura mas cálida pero el viaje con menor duración de los 5 no es el que posee la temperatura más fría, de hecho es el segundo con temperaturas más cálidas.  
* Todas las temperaturas varían mucho entre los diferentes viajes. Por ejemplo, la temperatura máxima del viaje con mayor duración es de aproximadamente 28°C mientras que la temperatura máxima del segundo viaje con mayor duración es de solamente 10°C, una diferencia de 18°C.
* Si tomamos en cuenta los dos puntos anteriores, podemos decir que no hay una correlación entre la temperatura y los viajes de mayor duración.

### 1.4.B) Visibilidad de los 5 Viajes con Mayor Duración

In [ ]:
top_dur_vis = top_dur[['duration_h', 'max_visibility_km', 'min_visibility_km', 'mean_visibility_km']]
top_dur_vis.set_index('duration_h', inplace=True)

Se muestran los 5 viajes con mayor duración (con sus respectivas duraciones, en horas, como label) ordenados de forma descendente junto con sus respectivas visibilidades máximas, visibilidades mínimas y visibilidades promedio.

In [ ]:
top_dur_vis_bar = top_dur_vis.plot.bar(figsize=(10,10), fontsize=20, rot=0);

En este plot se puede observar claramente que la visibilidad, tanto máxima, mínima y promedio, debe es igual o cercana a 16km. Solo un viaje de gran duración no tiene una visibilidad promedio de 16km pero su visibilidad máxima es de 16km. Estos datos indican que la visibilidad debe ser muy buena, alrededor de 16km., para que se produzcan viajes de gran duración.

### 1.4.C) Estación Climática de los 5 Viajes con Mayor Duracíon

In [ ]:
#Se muestran las duracion(en horas) y su respectiva estacion.
top_dur[['duration_h', 'estacion_clima']]

Las estaciones climáticas de los viajes de mayor duración, ordenados de mayor a menor, son:  
**1.** Verano  
**2.** Invierno  
**3.** Primavera  
**4.** Invierno  
**5.** Verano  

Dados estos datos se puede decir que:  
* La mayor duración de los 5 viajes se produce en Verano y la menor duración de los 5 viajes también se produce en Verano.  
* La distribución de los viajes en las estaciones climáticas es bastante pareja y solo falta Otoño.
* Dado esto podemos decir que no hay una correlación fuerte entre las estaciones climáticas y los viajes de mayor duración.

### 1.4.D) Conclución del Caso Particular

Con los análisis de los plots vistos se puede concluir que el requisito más importante para que se produzcan viajes largos es la buena visibilidad. Ademas, se podría agregar que como se requieren muchas horas para realizar estos viajes, estos se deberían producir en días no laborables o los que realizan estos viajes deberían ser turistas.

# 2)

In [ ]:
trips = pd.read_csv('trip.csv', low_memory=False)
#Se convierte los dates a datetime64[ns].
trips['start_date'] = pd.to_datetime(trips['start_date'])
trips['end_date'] = pd.to_datetime(trips['end_date'])

In [ ]:
#Se agrega una nueva columna date que coincide con weather.
trips['date'] = trips['start_date'].apply(lambda x: x.date())
#Se convierte date a datetime64[ns].
trips['date'] = pd.to_datetime(trips['date'])

In [ ]:
# chequeo tipos
trips.dtypes

In [ ]:
# ejemplo del uso de datetime con día actual
dia_actual = datetime.datetime.today()
dia_actual

In [ ]:
# lo paso a dia de la semana
dia_actual.weekday()

In [ ]:
# mejor en palabras que en números
calendar.day_name[dia_actual.weekday()]

In [ ]:
# función para convertir fecha a día de la semana.
def fecha_a_dia(fecha):
    return calendar.day_name[fecha.weekday()]

In [ ]:
#Se crean columnas con los dias de la semana.
trips['start_day_of_week'] = trips['start_date'].map(fecha_a_dia)
trips['end_day_of_week'] = trips['end_date'].map(fecha_a_dia)

In [ ]:
trips['horario_inicial'] = trips['start_date'].dt.time
trips['horario_inicial_float'] = trips['start_date'].dt.hour + trips['start_date'].dt.minute / 100

In [ ]:
trips['horario_final'] = trips['end_date'].dt.time
trips['horario_final_float'] = trips['end_date'].dt.hour + trips['end_date'].dt.minute / 100

In [ ]:
trips['duracion_viaje'] = trips['end_date'] - trips['start_date']

In [ ]:
# vista final de cómo quedó el dataframe
trips.sample(4)

## 2.0) ¿Viajes de menos de 3 minutos con misma estación de inicio y fin?
Una primera impresión es que hay viajes "ruidosos": duran menos de 3 minutos y las estaciones de inicio y fin
son las mismas. Esto nos dice que el usuario no tomó el viaje por algún motivo. Se van a filtrar esos viajes.

In [ ]:
viajes_ruidosos = trips[(trips['duracion_viaje'] <= '00:03:00') & (trips['start_station_id'] == trips['end_station_id'])]
trips = trips[-((trips['duracion_viaje'] <= '00:03:00') & (trips['start_station_id'] == trips['end_station_id']))]
# hay que hacer trips - viajes_ruidosos para que quede lindo pero requiere un comando (isin o algo asi). 
# no me salio bien asi que a lo ultimo lo modifico

In [ ]:
viajes_ruidosos.id.count()

In [ ]:
viajes_ruidosos.bike_id.value_counts().count()
# cantidad distintas de bicicletas dentro de esos 2601 viajes. Crei que iba a aportar información 
# para lo de las biciletas descompuestas pero no creo que ayude. Hay que analizar si es que varios de esos viajes
# con el mismo id de bicicleta se tomaron bastante pegados entonces si claramente las devolvieron porq estaban rotas

Se filtraron unos 2600 viajes. 

// acá iría la conclusión acerca de posibles bicicletas descompuestas

In [ ]:
# no es necesario pero lo dejo por las dudas, puede servir
# trips[-((trips['duracion_viaje'] < '1970-01-01 00:10:00') & (trips['start_station_id'] == trips['end_station_id']))]

In [ ]:
# ahora los datos quedan más limpios
trips.sample(4)

## 2.1) Análisis de viajes según día y horario

En estos primeros plots se analizará la relación general entre los viajes y el día/horario de la semana.

### 2.1.A) ¿El servicio se usa más en la semana o los fines de semana?

In [ ]:
trips['start_day_of_week'].value_counts().plot(kind='bar', rot=0, figsize=(10,8));
plt.title('Uso del servicio por dia');

In [ ]:
dias = trips[['start_day_of_week']]
dias_semana = dias[-(dias['start_day_of_week'] == "Saturday")]
dias_semana = dias[-(dias['start_day_of_week'] == "Sunday")]
dias_finde = dias[(dias['start_day_of_week'] == "Saturday") | (dias['start_day_of_week'] == "Sunday")]

In [ ]:
sizes = [dias_semana.start_day_of_week.count(), dias_finde.start_day_of_week.count()]
nombres = ['Semana', 'Fin de semana']

plt.figure(figsize=(6, 6))
plt.title('Porcentajes del uso del servicio')
plt.pie(sizes, labels=nombres, autopct='%1.1f%%', startangle=20, colors=['red', 'yellow'], explode=(0.1, 0))
plt.show()

Se ve que hay una diferencia drástica en el uso del servicio entre la semana y el fin de semana.

### 2.1.B) Promedio del uso del servicio por horario:

In [ ]:
semana_entera = trips[['start_day_of_week','horario_inicial_float', 'start_station_name', 'end_station_name']].round()
semana_entera

In [ ]:
semana = semana_entera[-(semana_entera['start_day_of_week'] == "Saturday")]
semana = semana_entera[-(semana_entera['start_day_of_week'] == "Sunday")]

In [ ]:
semana['apariciones'] = semana['start_day_of_week'].map(lambda x: 1) # seteo todas las rows con 1 para despues agrupar
horarios_semana = semana[['horario_inicial_float', 'apariciones']]
semana = semana.drop('apariciones', 1) # vuelvo a dejar el dt como antes
horarios_semana_contador = horarios_semana.groupby('horario_inicial_float').aggregate(sum)

In [ ]:
horarios_semana_contador.plot.bar(rot=0, figsize=(10,10), color='green');
plt.ylabel('Cantidad de viajes')
plt.xlabel('Horario de inicio del viaje')
plt.title('Promedio de uso del servicio por horario durante la semana')
plt.legend('')
plt.show()

Se aprecia que durante la semana los horarios pico son de 8 a 9 y de 18 a 19, particularmente cuando la gente va y cuando regresa al trabajo, escuela, etc.

In [ ]:
finde = semana_entera[(semana_entera['start_day_of_week'] == "Saturday") | (semana_entera['start_day_of_week'] == "Sunday")]

In [ ]:
finde['apariciones'] = finde['start_day_of_week'].map(lambda x: 1) # seteo todas las rows con 1 para despues agrupar
horarios_finde = finde[['horario_inicial_float', 'apariciones']]
finde = finde.drop('apariciones', 1) # vuelvo a dejar el dt como antes
horarios_finde_contador = horarios_finde.groupby('horario_inicial_float').aggregate(sum)

In [ ]:
horarios_finde_contador.plot.bar(rot=0, figsize=(10,10), color='green');
plt.ylabel('Cantidad de viajes')
plt.xlabel('Horario de inicio del viaje')
plt.title('Promedio de uso del servicio por horario durante el fin de semana')
plt.legend('')
plt.show()

Los horarios pico los fines de semana son de 11 a 16. También lo que se observa es la disminución del uso del servicio al llegar la noche a pesar de que sea fin de semana. En el siguiente plot se va a analizar esa comparación.

In [ ]:
viajes_en_hora_pico_finde = finde[((finde['horario_inicial_float'] >= 11) & (finde['horario_inicial_float'] <= 16))]
viajes_finde_noche = finde[((finde['horario_inicial_float'] >= 20) & (finde['horario_inicial_float'] <= 24))]

In [ ]:
sizes = [viajes_en_hora_pico_finde.start_day_of_week.count(), viajes_finde_noche.start_day_of_week.count()]
nombres = ['Hora pico', 'Durante la noche']

plt.figure(figsize=(6, 6))
plt.title('Uso del servicio los fines de semana')
plt.pie(sizes, labels=nombres, autopct='%1.1f%%', startangle=20, colors=['orange', 'violet'], explode=(0.1, 0))
plt.show()

## 2.2) Análisis de los horarios importantes

Se analizarán los destinos y trayectos más populares en los horarios críticos de la semana y fin de semana.

In [ ]:
viajes_en_hora_pico_semana = semana[((semana['horario_inicial_float'] >= 8) & (semana['horario_inicial_float'] <= 9)) 
                                   | ((semana['horario_inicial_float'] >= 18) & (semana['horario_inicial_float'] <= 19))]

In [ ]:
destinos_mas_populares_hora_pico_semana = viajes_en_hora_pico_semana['end_station_name'].value_counts().sort_values(ascending=False)
destinos_mas_populares_hora_pico_semana = destinos_mas_populares_hora_pico_semana.head(10)

In [ ]:
destinos_mas_populares_hora_pico_semana.plot(kind='bar', rot=45, figsize=(20,10));
plt.title('Destinos mas populares en hora pico semanal');

Estos son los 10 destinos más populares durante las horas pico en la semana. En primer lugar se destaca el Caltrain, donde una gran cantidad de gente se dirige luego de trabajar para regresar a sus casas. Las demás estaciones presentan áreas de mucho movimiento de gente: empresas, comercios, etc, y también se encuentra el puerto, por lo que lo más probable es que la gente se dirija allí para concurrir a trabajar.

In [ ]:
# función que dado un dataframe con un campo 'start_station_name' y otro 'end_station_name',
# devuelve un diccionario con los start_station como clave y como valor un diccionario con clave el end_station
# y valor la cantidad de viajes de ese trayecto. También devuelve una lista con el trayecto con mayor cantidad
# de viajes junto con el start y end station del mismo. Orden = O(n) siendo n la cantidad de rows del dataframe.
def contador_viajes(dataframe):
    cont_viajes = {}
    viaje_mas_popular = []
    viaje_mas_popular.append(0)
    viaje_mas_popular.append("")
    viaje_mas_popular.append("")
    for index,row in dataframe.iterrows():
        
        if row['start_station_name'] not in cont_viajes:
            cont_viajes[row['start_station_name']] = {}
           
        if row['end_station_name'] not in cont_viajes[row['start_station_name']]:
            cont_viajes[row['start_station_name']][row['end_station_name']] = 1
        else:
            cont_viajes[row['start_station_name']][row['end_station_name']] += 1
        
        if cont_viajes[row['start_station_name']][row['end_station_name']] > viaje_mas_popular[0]:
            viaje_mas_popular[0] = cont_viajes[row['start_station_name']][row['end_station_name']]
            viaje_mas_popular[1] = row['start_station_name']
            viaje_mas_popular[2] = row['end_station_name']
    
    return cont_viajes,viaje_mas_popular

In [ ]:
contador_de_viajes_hora_pico_semana,viaje_mas_popular_hora_pico_semana = contador_viajes(viajes_en_hora_pico_semana)
viaje_mas_popular_hora_pico_semana

Con esto se ve que el viaje más realizado en hora pico durante la semana es el trayecto San Francisco Caltrain 2 (330 Townsend) - Townsend at 7th.

In [ ]:
destinos_mas_populares_hora_pico_finde = viajes_en_hora_pico_finde['end_station_name'].value_counts().sort_values(ascending=False)
destinos_mas_populares_hora_pico_finde = destinos_mas_populares_hora_pico_finde.head(10)

In [ ]:
destinos_mas_populares_hora_pico_finde.plot(kind='bar', rot=45, figsize=(20,10));
plt.title('Destinos mas populares en hora pico fin de semana');

Aqui se encuentran los 10 destinos más populares para las horas pico del fin de semana. Se puede apreciar que los mismos se caracterizan por ser lugares muy atractivos para pasear y hacer actividades de ocio.

In [ ]:
contador_de_viajes_hora_pico_finde,viaje_mas_popular_hora_pico_finde = contador_viajes(viajes_en_hora_pico_finde)
viaje_mas_popular_hora_pico_finde

Con esto se ve que el viaje más realizado en hora pico durante el fin de semana es el trayecto Harry Bridges Plaza (Ferry Building) - Embarcadero at Sansome. Además estos dos son los destinos más concurridos las horas pico de los fines de semana. Un posible uso de esta información podría ser para fines comerciales, ya que esto da la pauta que por esta zona es donde más concentración de gente se encuentra. 

In [ ]:
viernes_y_sab = semana_entera[(semana_entera['start_day_of_week'] == "Saturday") | (semana_entera['start_day_of_week'] == "Friday")]
viernes_y_sab_noche = viernes_y_sab[(viernes_y_sab['horario_inicial_float'] >= 20) & (viernes_y_sab['horario_inicial_float'] <= 24)]

In [ ]:
destinos_mas_populares_viernes_y_sab_noche = viernes_y_sab_noche['end_station_name'].value_counts().sort_values(ascending=False)
destinos_mas_populares_viernes_y_sab_noche = destinos_mas_populares_viernes_y_sab_noche.head(10)

In [ ]:
destinos_mas_populares_viernes_y_sab_noche.plot(kind='bar', rot=45, figsize=(20,10));
plt.title('Destinos mas populares viernes y sabado por la noche');

Estos son los destinos más frecuentados los viernes y sábados por la noche. Se analizó aparte del domingo ya que estos son los días que al día siguiente por lo general no se trabaja/concurre a estudiar, por lo que la gente se podría dormir más tarde y planear otro tipo de salida. Se destacan entre los lugares más frecuentados las estaciones Powell Street BART y Market at 4th, las cuales están rodeadas de shoppings, restaurantes y demás.

## 2.3) Análisis relacionado a las duraciones de los viajes

### 2.3.A) ¿Viajes más largos en la semana o el fin de semana?

In [ ]:
semana_entera_con_duracion = trips[['start_day_of_week','horario_inicial_float', 'start_station_name', 'end_station_name','duracion_viaje','duration']].round()
semana_entera_con_duracion

### 2.3.) ¿Viajes de más de 8 horas de duración?
Se pueden esperar viajes muy largos de a lo sumo 8 horas de alguien que decidió recorrer varios puntos de la ciudad y aparte en el medio ir parando, pero ya cuando se excede esto hasta casos que incluso superan un día de uso creemos que hubo datos mal cargados o algún otro problema como la incorrecta devolución de la bicicleta. Por eso se van a filtrar esos viajes.

In [ ]:
viajes_larguisimos = semana_entera_con_duracion[semana_entera_con_duracion['duracion_viaje'] > "08:00:00"]
semana_entera_con_duracion = semana_entera_con_duracion[-(semana_entera_con_duracion['duracion_viaje'] > "08:00:00")]
viajes_larguisimos.sample(4)

In [ ]:
viajes_larguisimos.start_day_of_week.count()

Se filtraron unos 1884 viajes.

In [ ]:
semana_con_duracion = semana_entera_con_duracion[-(semana_entera_con_duracion['start_day_of_week'] == "Saturday")]
semana_con_duracion = semana_entera_con_duracion[-(semana_entera_con_duracion['start_day_of_week'] == "Sunday")]

In [ ]:
finde_con_duracion = semana_entera_con_duracion[(semana_entera_con_duracion['start_day_of_week'] == "Saturday") | (semana_entera_con_duracion['start_day_of_week'] == "Sunday")]

In [ ]:
semana_con_duracion.duration.mean() / 60 # resultado en minutos

In [ ]:
finde_con_duracion.duration.mean() / 60 # resultado en minutos

Los viajes en promedio duran más los fines de semana. Esto tiene lógica ya que los fines de semana la gente puede usar más tiempo para pasear con la bicileta sin tener un destino en particular cuando durante la semana la gente busca ir al trabajo o lugar donde ejercer sus obligaciones.
Igualmente hay que considerar que esta estadística podría estar dañada por muchos viajes "largos" en el fin de semana, por ejemplo de más de dos horas, que harían subir el promedio general, sin haber tantos viajes con un horario parecido al promedio calculado. A continuación se analizará eso:

In [ ]:
semana_con_duracion[semana_con_duracion['duracion_viaje'] > "02:00:00"].start_day_of_week.count()

In [ ]:
finde_con_duracion[finde_con_duracion['duracion_viaje'] > "02:00:00"].start_day_of_week.count()

Se observa que tanto en la semana como los fines de semana hay viajes de más de 2 horas, y que ya que la semana tiene más días que el fin de semana y la estadística muestra que son casi el doble de viajes, en promedio esos viajes largos se dan por igual todos los días, por lo que el probema de que haya viajes largos no afecta el resultado de la estadística anterior.

## 2.4) Trayectos interesantes

A continuación se analizarán los trayectos más frecuentes según un criterio de duración y demás fitros.

In [ ]:
finde_viajes_largos = finde_con_duracion[finde_con_duracion['duracion_viaje'] > "00:30:00"]

In [ ]:
contador_de_viajes_largos_finde,viaje_largo_mas_popular_finde = contador_viajes(finde_viajes_largos)
viaje_largo_mas_popular_finde.sample(4)

Con esto se ve que los viajes mayores a 30 minutos los fines de semana son tanto de inicio como fin la estación Harry Bridges Plaza (Ferry Building). El objetivo de este análisis fue encontrar el trayecto más popular de los fines de semana que sea orientado a pasear (por eso el filtro de la duración). El resultado muestra que el viaje más realizado es empezar en dicha estación, recorrer por más de 30 minutos, y luego volver a esta misma dejar la bicicleta.

In [ ]:
harry_harry = finde_viajes_largos[(finde_viajes_largos['start_station_name'] == 'Harry Bridges Plaza (Ferry Building)') & (finde_viajes_largos['end_station_name'] == 'Harry Bridges Plaza (Ferry Building)')]
harry_harry.describe()

Como se puede apreciar, estos viajes se caracterizan por su duración promedio de 2 horas y media y que en promedio salen a las 13hs. Esto da la pauta de qué elige la mayoría de gente a la hora de pasear por un largo rato aprovechando el clima de recién entrada la tarde.

In [ ]:
semana_viajes_cortos = semana_con_duracion[(semana_con_duracion['duracion_viaje'] > "00:08:00") & (semana_con_duracion['duracion_viaje'] < "00:20:00")]

In [ ]:
contador_de_viajes_cortos_semana,viaje_corto_mas_popular_semana = contador_viajes(semana_viajes_cortos)
viaje_corto_mas_popular_semana.sample(4)

In [ ]:
steuart_caltrain = semana_viajes_cortos[(semana_viajes_cortos['start_station_name'] == 'Steuart at Market') & (semana_viajes_cortos['end_station_name'] == 'San Francisco Caltrain (Townsend at 4th)')]
steuart_caltrain.describe()

Con esto se ve que el viaje entre 8 y 20 minutos más popular los días de semana es el trayecto Steuart at Market - San Francisco Caltrain (Townsend at 4th). El objetivo de este análisis fue encontrar el trayecto con una duración normal que más se haga considerando el tráfico que presenta un día semanal. Una posible conclusión debido a la hora promedio de comienzo de los viajes (15hs) es que la gente al terminar de trabajar se dirige desde Steuart at Market (un lugar con mucho movimiento por lo que deducimos que mucha gente trabaja por esa zona) en bicicleta hasta el Caltrain, y toma este transporte para volver a sus casas.

# 3)

In [ ]:
stations = pd.read_csv('../data/station.csv', low_memory = False)

stations.rename(columns = {'installation_date':'date',
                           'long'             :'lon'
                          }, inplace = True)

# Cambio el formato de las fechas
stations['date'] = pd.to_datetime(stations['date'])

In [ ]:
trips = pd.read_csv('../data/trip.csv', low_memory = False)

#Eliminamos este dato ya que no sera relevante para estos analisis
del(trips['zip_code'])

trips.rename(columns = {'start_date'        :'s_date' ,
                        'end_date'          :'e_date' ,
                        'start_station_name':'ss_name',
                        'start_station_id'  :'ss_id'  ,
                        'end_station_name'  :'es_name',
                        'end_station_id'    :'es_id'  ,
                        'subscription_type' :'subs'
                       }, inplace = True)

# Cambio el formato de las fechas y tiempos de incio y fin de cada viaje
trips['s_date'] = pd.to_datetime(trips['s_date'], format = '%m/%d/%Y %H:%M')
trips['e_date'] = pd.to_datetime(trips['e_date'], format = '%m/%d/%Y %H:%M')

In [ ]:
#Quitamos los viajes de duracion menor o igual a 3 minutos (180 segundos) y que de la misma estacion,
#ya que pueden ser rutinas de mantenimiento o reparacion, etc
trips = trips[-((trips['duration'] <= 180) & \
                (trips['ss_id'] == trips['es_id']))]

# Ordeno los viajes por id
trips = trips.sort_values(by = 'id')

## 3.1) Suscriptores vs Clientes

Quienes utilizan mas el servicio? Suscriptores o clientes comunes?

In [ ]:
subs = trips.groupby('subs').count()[['id']]

plt.figure(figsize = (4, 4))
plt.pie(subs.values,
        explode = (0.1, 0),
        labels = subs.index.values,
        startangle = -25,
        colors = ['lightgrey', 'gold'],
        autopct = '%1.1f%%'
       )
plt.savefig('visu_subs_vs_cust.png')
plt.show()

Aqui se ve que la mayoria de los usuarios del servicio, estan suscriptos a este mismo.

## 3.2) Uso de Estaciones por candtidad de bicicletas totales

In [ ]:
trips_13_15 = trips
trips_2013  = trips[          trips['s_date']  < '2014'                    ].reset_index()
trips_2014  = trips[('2014' < trips['s_date']) & (trips['s_date'] < '2015')].reset_index()
trips_2015  = trips[ '2015' < trips['s_date']                              ].reset_index()

viajes_por_anio = [trips_13_15, trips_2013, trips_2014, trips_2015]
anios = ['2013-2015', '2013', '2014', '2015']

#Calculamos la suma de bicicletas que partieron de las estaciones usadas en cada año
for i in range(len(viajes_por_anio)):
    viajes_por_anio[i] = viajes_por_anio[i].groupby('ss_name')['id'].count().sort_values(ascending = False)

In [ ]:
plt.figure(figsize = (20, 7))

for i in range(len(viajes_por_anio)):
    plt.subplot(221 + i);
    plt.title(anios[i]);
    plt.xlabel('Cantidad de bicicletas totales');
    plt.ylabel('Estacion de inicio');
    plt.barh([c for c in range(5)],
             #[c for c in viajes_por_anio[i][:5].values],
             viajes_por_anio[i][:5].values,
             tick_label = [c for c in viajes_por_anio[i][:5].index.values])
    plt.savefig('viajes_' + anios[i] + '.png')

plt.tight_layout()

.

## 3.3) Visualizando con mapas

Ahora vamos a visualizar las estaciones y sus frecuencias de uso en mapas.

In [ ]:
from math import pi as PI

#Estas son algunas funciones que nos van a ayudar a manejar basemap para centrar los mapas en una cierta zona
def distancia_grados(dist):
    return (180 * dist) / (6371 * PI)

def distancia_km(angulo):
    return angulo * 6371 * PI / 180

def mean(a, b):
    return (a + b) / 2

In [ ]:
#Calculamos la cantidad de bicicletas por cada estacion
bicis_por_estacion = trips.groupby('ss_id').count()[['ss_name']].reset_index()

bicis_por_estacion.rename(columns = {'ss_id' :'id',
                                     'ss_name':'count'
                                    }, inplace = True)

#Agregamos el nombre de la estacion, la ciudad en donde queda y sus coordenadas
estaciones = pd.merge(stations.drop(['date', 'dock_count'], 1), bicis_por_estacion, on = 'id')

### 3.3.A) Frecuencia de uso de Estaciones por ciudad

Lo que se quiere ver ahora es, como se distribuyen los viajes realizados entre las estaciones de cada ciudad.
(se utiliza la cantidad de bicicletas que parten de una estacion como medida de frecuencia de uso de una estacion)

In [ ]:
#Separamos las estaciones por ciudad
sf = estaciones[estaciones['city'] == 'San Francisco'].reset_index(drop = True)
sj = estaciones[estaciones['city'] == 'San Jose'     ].reset_index(drop = True)
mv = estaciones[estaciones['city'] == 'Mountain View'].reset_index(drop = True)
pa = estaciones[estaciones['city'] == 'Palo Alto'    ].reset_index(drop = True)
rc = estaciones[estaciones['city'] == 'Redwood City' ].reset_index(drop = True)

ciudades = [sf, sj, mv, pa, rc, estaciones]
nombres  = ['San Francisco', 
            'San Jose', 
            'Mountain View', 
            'Palo Alto', 
            'Redwood City', 
            'Bay']
ciudades_cant_estaciones = []
ciudades_bicis = []

#Listas con latitudes y longitudes de cada ciudad
lats = []
lons = []

offsets = []

#Para cada ciudad, vemos su cantidad de estaciones y la cantidad total de bicicletas
for ciudad in ciudades:
    ciudades_cant_estaciones.append(ciudad.shape[0])
    ciudades_bicis.append(ciudad['count'].sum())
    #Para incluir a todas las estaciones (en la visualizacion), tomamos la media entre la coordenada maxima y minima
    lat_max, lat_min = ciudad['lat'].max(), ciudad['lat'].min()
    lon_max, lon_min = ciudad['lon'].max(), ciudad['lon'].min()
    lats.append(mean(lat_max, lat_min))
    lons.append(mean(lon_max, lon_min))
    offsets.append(max((lat_max - lat_min) * 0.6, (lon_max - lon_min) * 0.6))

#La cantidad de estaciones se va a utilizar para escalar a los tamaños de las estaciones en la visualizacion

In [ ]:
#Seteamos algunas variables para los mapas
proyeccion = 'merc'
resolucion = 'l'
epsg = 4326
color = 'tomato'
servicio = 'NatGeo_World_Map'
x_tam, y_tam = 500, 500 #1000, 1000
marcador = 'o'

In [ ]:
plt.figure(figsize = (20, 20))

for i in range(len(ciudades)):
    plt.subplot(331 + i);
    plt.title(nombres[i]);
    mapa = Basemap(projection = proyeccion, 
                   resolution = resolucion, 
                   epsg = epsg, 
                   llcrnrlat = lats[i] - offsets[i], 
                   llcrnrlon = lons[i] - offsets[i], 
                   urcrnrlat = lats[i] + offsets[i], 
                   urcrnrlon = lons[i] + offsets[i])
    
    #Ploteamos las estaciones e indicamos frecuencia de uso segun la cantidad de viajes inciados desde esa estacion
    #Aqui se escalan los tamaños de las estaciones dependiendo de la ciudad que se muestre
    mapa.scatter(ciudades[i]['lon'].values, 
                 ciudades[i]['lat'].values, 
                 marker = marcador, 
                 c = color, 
                 s = ciudades[i]['count'].apply(lambda x:x/ciudades_cant_estaciones[i]));
    
    mapa.arcgisimage(service = servicio, 
                     xpixels = x_tam, 
                     ypixels = y_tam);
    
    plt.savefig(nombres[i] + '_map.png');
    

### 3.3.B) Comparando todas las ciudades

In [ ]:
#Buscando la coordenada media entre todas las estaciones para centrar al mapa
max_lat, max_lon = station_bikes.max()[['lat', 'lon']]
min_lat, min_lon = station_bikes.min()[['lat', 'lon']]
lat, lon = mean(max_lat, min_lat), mean(max_lon, min_lon)

plt.figure(figsize = (10, 10))
offset = distancia_grados(35)
mapa = Basemap(projection = proyeccion, 
               resolution = resolucion, 
               epsg = epsg, 
               llcrnrlat = lat - offset, 
               llcrnrlon = lon - offset, 
               urcrnrlat = lat + offset, 
               urcrnrlon = lon + offset)
  
mapa.scatter(lons, 
             lats, 
             s = ciudades_bicis, 
             marker = marcador, 
             c = color)

mapa.arcgisimage(service = servicio, 
                 xpixels = x_tam,  
                 ypixels = y_tam)

plt.savefig('bahia_viajes_ciudades.png')
plt.show()

## 3.4) Comparando viajes por ciudad

In [ ]:
plt.figure(figsize = (6, 6))
plt.pie(ciudades_bicis[:-1:], 
        labels = nombres[:-1:], 
        autopct = '%1.1f%%', 
        explode = (0, .15, .25, .35, .45))
plt.show()

La gran parte de los viajes parten de estaciones ubicadas en la ciudad de San Francisco

In [ ]:
#Dataframe con los nombres de todas las estaciones y sus coordenadas
ss_location = stations[['name', 'lat', 'lon', 'city']]
ss_location.rename(columns = {'name':'ss_name', 
                              'lat' :'s_lat', 
                              'lon' :'s_lon', 
                              'city':'s_city'
                             }, inplace = True)

es_location = stations[['name', 'lat', 'lon', 'city']]
es_location.rename(columns = {'name':'es_name', 
                              'lat' :'e_lat', 
                              'lon' :'e_lon', 
                              'city':'e_city'
                             }, inplace = True)

#Estos dataframes seran combinados con los de trayectos por la columna que tengan en comun
#En este caso, se quiere obtener un dataframe con 2 estaciones (inicial y final) con sus coordenadas y viajes

## 3.5) Trayectos mas frecuentes

### 3.5.A) Trayectos por ciudad

Lo que queremos lograr ahora es, ver cuales son los viajes entre estaciones mas utilizados.

Ademas de calcular los trayectos hay que juntar los que estan repetidos. Los trayectos con mas repeticiones seran considerados como los mas frecuentes.

In [ ]:
trayectos_frec = trips[['id', 'ss_name', 'es_name']].groupby(['es_name', 'ss_name'], as_index = False).count()

trayectos_frec.rename(columns = {'id':'count'}, inplace = True)

In [ ]:
def reducir_trayectos(trayectos_frec):
    #El trayecto A -> B es el mismo que B -> A
    #Esta funcion junta las repeticiones de estos trayectos
    l = [] 
        #Lista con indice del dataframe del trayecto eliminado
    for i in range(len(trayectos_frec)): 
            #Por cada trayecto
        if i not in l:
            #Si esta en el dataframe
            prim = trayectos_frec.loc[i]
                #Obtenemos el trayecto actual
            for j in range(i, len(trayectos_frec)): 
                #Buscamos en lo que resta del dataframe
                if j not in l: 
                    #Si el trayecto que queremos ver, esta en el dataframe
                    seg = trayectos_frec.loc[j] 
                        #Lo guardamos
                    if prim['ss_name'] == seg['es_name'] and prim['es_name'] == seg['ss_name']: 
                        #Si coinciden los trayectos
                        trayectos_frec.loc[i]['count'] += trayectos_frec.loc[j]['count']
                            #Sumamos los trayectos
                            #Nosotros consideramos que los trayectos de A a B son iguales que los de B a A
                            #Pero en el dataframe no estan asi, por eso sumamos los viajes
                        trayectos_frec.drop(j, inplace = True)
                            #Quitamos al segundo trayecto del dataframe
                        l.append(j)
                            #Lo colocamos en la lista, para que no halla error luego
    return trayectos_frec

#Esta funcion, puede tomar un tiempo
#Si saben como hacerlo mas eficiente, modifiquenlo pls

tf = reducir_trayectos(trayectos_frec)

# Añadimos al dataframe, las ubicaciones de cada estacion
tf = pd.merge(tf, ss_location, on = 'ss_name')
tf = pd.merge(tf, es_location, on = 'es_name')

In [ ]:
#Este diccionario se va a contener, por ciudad:
#    . lista de tuplas del estilo ([lons], [lats], color) que tienen las coord de estaciones de un trayecto y su color
#    . lista de los trayectos de la ciudad
#respectivamente. Esto es para automatizar y tambien para colocar a todos los mapas en un subplot (de matplotlib)
viajes = {'San Francisco': [[], []], 
          'San Jose'     : [[], []], 
          'Redwood City' : [[], []], 
          'Mountain View': [[], []], 
          'Palo Alto'    : [[], []]
         }

for ciudad in viajes:
    #Filtramos los trayectos por ciudad
    #Aqui se quitan los trayectos con estaciones en distintas ciudades
    viajes[ciudad][1] = tf[(tf['s_city'] == ciudad) & \
                           (tf['e_city'] == ciudad)].drop(['s_city', 'e_city'], 1).reset_index(drop = True)
    
    #Calculamos la cantidad maxima de viajes por cada trayecto en cada ciudad
    #Esto es para luego asignar colores a trayectos con mas o menos frecuencia
    max_val = 0
    for indice in range(len(viajes[ciudad][1])):
        trayecto = viajes[ciudad][1].loc[indice]
        if trayecto['count'] > max_val: 
            max_val = trayecto['count']
    
    #Colocamos longitud y latitud de un trayecto ademas de su frecuencia convertida a un numero en [0;255]
    for indice in range(len(viajes[ciudad][1])):
        trayecto = viajes[ciudad][1].loc[indice]
        #Mapeo de la frecuencia del trayecto al intervalo [0;255]
        color = int(255 / float(max_val) * trayecto['count'])
        color = abs(255 - color)
        viajes[ciudad][0].append(([trayecto.['s_lon'], trayecto.['e_lon']],
                                  [trayecto.['s_lat'], trayecto.['e_lat']],
                                  color)
                                )
    #Ordenamos los viajes mas frecuentes al final, asi son mostrados por encima de los trayectos menos frecuentes
    viajes[ciudad][0].sort(key = lambda x: -x[2])

In [ ]:
#Ahora visualizamos los trayectos por ciudad

plt.figure(figsize = (20, 13))

for i in range(len(ciudades)):
    plt.subplot(231 + i);
    plt.title('Viajes en ' + nombres[i]);
    offset = distancia_grados(2.5)
    mapa = Basemap(projection = proyeccion, 
                   resolution = resolucion, 
                   epsg = epsg, 
                   llcrnrlat = lats[i] - offset, 
                   llcrnrlon = lons[i] - offset, 
                   urcrnrlat = lats[i] + offset, 
                   urcrnrlon = lons[i] + offset)
    
    for trayecto in viajes[nombres[i]][0]:
        #Convertimos el color a un formato RGBA valido
        color = trayecto[2]
        color = '#%02X%02X%02X' % (color, color, color)
        #Ploteamos el trayecto
        mapa.plot(trayecto[0], 
                  trayecto[1], 
                  'o-', 
                  color = color)
    
    mapa.arcgisimage(service = servicio, 
                     xpixels = x_tam, 
                     ypixels = y_tam);
    
    plt.savefig(nombres[i] + '_map_viajes.png');

### 3.5.B) Todos los trayectos

In [ ]:
#Esto falta editarlo un poco mas, para que quede lindo (ahora son solo lineas azules)
#Es un mapa de toda la bahia con todos los viajes realizados (conecta estacion inicial con final por cada viaje)

plt.figure(figsize = (10, 10))
offset = distancia_grados(35)
mapa = Basemap(projection = proyeccion, 
               resolution = resolucion, 
               epsg = epsg, 
               llcrnrlat = lat - offset, 
               llcrnrlon = lon - offset, 
               urcrnrlat = lat + offset, 
               urcrnrlon = lon + offset)

mapa.plot(tf['s_lon'], 
          tf['s_lat'], 
          'o-')

mapa.arcgisimage(service = servicio, 
                 xpixels = x_tam, 
                 ypixels = y_tam)
plt.show()